# Gymnasium integration
[Gymnasium](https://gymnasium.farama.org/) is an API standard for reinforcement learning with a diverse collection of reference environments. Stormvogel supports some integration with gymnasium. In particular, you can construct explicit models from the gymnasium environmnents under Gymnasium's [ToyText](https://gymnasium.farama.org/environments/toy_text/) (except Blackjack). Let us create one of these environments, called FrozenLake.

Our agent wants to get to the present. Currently, it just chooses a random action.

In [33]:
import gymnasium as gym
from stormvogel.extensions.gym_grid import *

env = gym.make("FrozenLake-v1", render_mode="rgb_array", is_slippery=False)  # Set `is_slippery=True` for stochastic behavior
embed_video(create_video(env, "./video", None))

We can convert it into an explicit MDP as follows. Each state has a label that relates to the coordinates of the tile.

In [35]:
from stormvogel.show import show, Layout
sv_model = gymnasium_to_stormvogel(env)
vis = show(sv_model, layout=Layout("layouts/frozenlake.json"))

Output()

Output()

Now, let's do some model checking to calculate a strategy to solve the puzzle. We will tell the model checker to maximize the probability of getting to the target state (the present).

In [36]:
from stormvogel.model_checking import model_checking
target = get_target_state(env)
res = model_checking(sv_model, f'Pmax=? [F "target"]')
vis2 = show(sv_model, result=res, layout=Layout("layouts/frozenlake.json"))

Output()

Output()

Let's highlight the path to see what the scheduler is doing.

In [37]:
from stormvogel.simulator import simulate_path
path = simulate_path(sv_model, scheduler=res.scheduler, steps=20)
vis2.highlight_path(path, color="orange")

Alternatively, we can show what our scheduler does in the frozen lake environment itself.

In [38]:
from stormvogel.extensions.gym_grid import *
embed_video(create_video_scheduler(sv_model, env, "./video", res.scheduler))